In [17]:
import pandas as pd
import numpy as np
import tejapi

# Import Market Cap Data

In [18]:
tejapi.ApiConfig.api_base = "http://10.10.10.66"
tejapi.ApiConfig.api_key = "rCQ1mgb1y8bIgShFl5lLinrXWhjwvD"

In [3]:
marketCap = tejapi.get('TWN/AIFINQ',
                        acc_code = 'MV',
                        # coid = coidList,
                        paginate=True)

In [3]:
marketCap = pd.read_csv("data/marketCap.csv")
marketCap.rename(columns={'coid': 'company_id', 'mdate': 'date', 'acc_value': 'marketCap'}, inplace=True)
marketCap

,company_id,date,acc_code,marketCap
0,1101,2008-03-01 00:00:00+00:00,MV,1.913447e+08
1,1101,2008-06-01 00:00:00+00:00,MV,1.331941e+08
2,1101,2008-09-01 00:00:00+00:00,MV,6.139906e+07
3,1101,2008-12-01 00:00:00+00:00,MV,8.872412e+07
4,1101,2009-03-01 00:00:00+00:00,MV,9.201629e+07
...,...,...,...,...
104748,Y9999,2021-12-01 00:00:00+00:00,MV,5.487441e+10
104749,Y9999,2022-03-01 00:00:00+00:00,MV,5.365555e+10
104750,Y9999,2022-06-01 00:00:00+00:00,MV,4.497718e+10
104751,Y9999,2022-09-01 00:00:00+00:00,MV,4.013845e+10


In [4]:
marketCap.company_id = marketCap.company_id.apply(lambda x: str(x))

# Merge SASB industry category

In [5]:
sasb = pd.read_csv("data/SASB.csv")
sasb.company_id = sasb.company_id.apply(lambda x: str(x))

In [6]:
df = pd.merge(marketCap, sasb, on='company_id', how="left").dropna()

In [8]:
df["date"] = pd.to_datetime(df["date"]).dt.date
df

,company_id,date,acc_code,marketCap,company_name,SASB
0,1101,2008-03-01,MV,191344719.0,台泥,提煉與礦產加工
1,1101,2008-03-01,MV,191344719.0,台泥,提煉與礦產加工
2,1101,2008-03-01,MV,191344719.0,台泥,提煉與礦產加工
3,1101,2008-03-01,MV,191344719.0,台泥,提煉與礦產加工
4,1101,2008-03-01,MV,191344719.0,台泥,提煉與礦產加工
...,...,...,...,...,...,...
570197,9962,2022-09-01,MV,1326234.0,有益,提煉與礦產加工
570198,9962,2022-09-01,MV,1326234.0,有益,提煉與礦產加工
570199,9962,2022-09-01,MV,1326234.0,有益,提煉與礦產加工
570200,9962,2022-09-01,MV,1326234.0,有益,提煉與礦產加工


In [10]:
df_m = df.pivot_table(index='date',columns='company_id')

In [11]:
df_m.columns = df_m.columns.get_level_values(1)

In [ ]:
sasbDict = sasb.drop_duplicates(subset=['company_id'], keep='last')[['company_id', 'SASB']]
sasbDict = sasbDict.sort_values('company_id')
sasbDict

In [12]:
df_weight = pd.DataFrame()
for ind in [i for i in sasbDict.SASB.unique()]:
    lst = sasbDict[sasbDict['SASB']==ind]['company_id'].tolist()
    data = df_m[df_m.columns.intersection(lst)]
    data = data.div(data.sum(axis=1), axis=0)
    df_weight = pd.concat([df_weight, data], axis=1)


In [13]:
df_weight

company_id,1101,1102,1103,1104,1108,1109,1110,1516,1584,1802,...,6016,6020,6021,6023,6024,6026,6027,6028,6592,9941
date,,,,,,,,,,,,,,,,,,,,,
2008-03-01,0.085625,0.066450,0.008253,0.005955,0.002318,0.002515,0.003621,0.000281,0.000690,0.030057,...,0.003573,0.001871,0.001946,0.001669,NaN,NaN,NaN,NaN,NaN,0.001768
2008-06-01,0.065849,0.061515,0.007191,0.007035,0.001949,0.002280,0.003147,0.000532,0.000521,0.028832,...,0.002584,0.001269,0.001480,0.002111,0.000884,NaN,NaN,NaN,NaN,0.001895
2008-09-01,0.041200,0.054464,0.006398,0.006362,0.001616,0.002231,0.003019,0.000629,0.000315,0.023553,...,0.002613,0.001541,0.001482,0.002922,0.000986,NaN,NaN,NaN,NaN,0.002340
2008-12-01,0.065371,0.060763,0.006819,0.005673,0.001789,0.002532,0.003877,0.001157,0.000303,0.026867,...,0.002804,0.001514,0.001128,0.003591,0.001112,NaN,NaN,NaN,NaN,0.002288
2009-03-01,0.067433,0.062455,0.007102,0.005797,0.001827,0.002780,0.003550,0.000934,0.000422,0.025723,...,0.003250,0.001925,0.001396,0.003660,0.001307,NaN,NaN,NaN,NaN,0.003257
2009-06-01,0.062914,0.061747,0.006597,0.005476,0.001744,0.002419,0.003986,0.000800,0.000366,0.025753,...,0.003011,0.001491,0.001500,0.002559,0.001061,NaN,NaN,NaN,NaN,0.003096
2009-09-01,0.065929,0.059626,0.006730,0.005360,0.001875,0.002382,0.003697,0.000692,0.000417,0.028412,...,0.002995,0.001712,0.001627,0.002052,0.001031,NaN,NaN,NaN,NaN,0.002875
2009-12-01,0.059429,0.054848,0.006899,0.005611,0.001923,0.002347,0.003447,0.000660,0.000533,0.027939,...,0.003286,0.001738,0.002350,0.002297,0.001153,NaN,NaN,NaN,NaN,0.003382
2010-03-01,0.053160,0.049902,0.006057,0.005291,0.001774,0.002304,0.003531,0.000852,0.000565,0.036493,...,0.002907,0.001572,0.002020,0.002193,0.001096,NaN,NaN,NaN,NaN,0.003586


In [16]:
df_weight.to_csv("output/rriSASBWeight.csv")

In [15]:
df_weight['2330']

date
2008-03-01    0.155764
2008-06-01    0.174450
2008-09-01    0.175826
2008-12-01    0.194917
2009-03-01    0.189331
2009-06-01    0.169097
2009-09-01    0.165790
2009-12-01    0.144918
2010-03-01    0.142345
2010-06-01    0.152184
2010-09-01    0.142538
2010-12-01    0.151194
2011-03-01    0.159796
2011-06-01    0.166257
2011-09-01    0.193903
2011-12-01    0.209871
2012-03-01    0.202846
2012-06-01    0.210549
2012-09-01    0.221808
2012-12-01    0.236876
2013-03-01    0.236710
2013-06-01    0.250877
2013-09-01    0.235606
2013-12-01    0.235934
2014-03-01    0.241466
2014-06-01    0.231681
2014-09-01    0.234629
2014-12-01    0.257808
2015-03-01    0.258913
2015-06-01    0.257616
2015-09-01    0.276567
2015-12-01    0.290433
2016-03-01    0.307756
2016-06-01    0.304767
2016-09-01    0.320609
2016-12-01    0.320268
2017-03-01    0.308606
2017-06-01    0.309551
2017-09-01    0.315377
2017-12-01    0.330250
2018-03-01    0.344739
2018-06-01    0.306904
2018-09-01    0.377822
2018-1

# Merge TSE industry category

In [20]:
data = tejapi.get('TWN/AIND')

In [22]:
data

,coid,mdate,mkt,elist_day1,tseid,ind,ind1,emm,ind_ban,isin,...,tejinm1_e,tejico2,tejinm2_c,tejinm2_e,tejico3,tejinm3_c,tejinm3_e,tejico4,tejinm4_c,tejinm4_e
None,,,,,,,,,,,,,,,,,,,,,
0,000116,2023-01-01 00:00:00+00:00,,1996-06-28 00:00:00+00:00,000116,30,17,12,ID 03098908,TW0006014004,...,Securities,M30A,證券,Securities Firm,M30A,證券,Securities Firm,M2800,金融業,Financial
1,000930,2023-01-01 00:00:00+00:00,,1995-12-27 00:00:00+00:00,000930,30,17,12,ID 22955774,TW0006009004,...,Securities,M30A,證券,Securities Firm,M30A,證券,Securities Firm,M2800,金融業,Financial
2,000960,2023-01-01 00:00:00+00:00,,1995-12-12 00:00:00+00:00,000960,30,17,12,ID 22957301,TW0006007008,...,Securities,M30A,證券,Securities Firm,M30A,證券,Securities Firm,M2800,金融業,Financial
3,0009A0,2023-01-01 00:00:00+00:00,,1994-12-06 00:00:00+00:00,0009A0,30,17,12,ID 23113343,TW0006001001,...,Securities,M30A,證券,Securities Firm,M30A,證券,Securities Firm,M2800,金融業,Financial
4,1101,2023-01-01 00:00:00+00:00,TSE,1962-02-09 00:00:00+00:00,1101,11,01,12,ID 11913502,TW0001101004,...,Cement,M11A,水泥製造,Cement Manufacturing,M11A,水泥製造,Cement Manufacturing,M1100,水泥工業,Cement
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3147,9962,2023-01-01 00:00:00+00:00,OTC,2006-07-10 00:00:00+00:00,9962,20,10,12,ID 96862363,TW0009962001,...,Iron and Steel,M20A,金屬基本,Metal,M20A6,不鏽鋼,Stainless Steel,M2000,鋼鐵工業,Iron & Steel
3148,9965,2023-01-01 00:00:00+00:00,,2006-12-29 00:00:00+00:00,9965,26,15,12,ID 22765839,TW0009965004,...,Shipping and Trans.,M26C,貨運倉儲業,Freight Warehouse,M26C,貨運倉儲業,Freight Warehouse,M2600,航運業,Shipping & Trans.
3149,L2301,2023-01-01 00:00:00+00:00,,1983-01-26 00:00:00+00:00,L2301,23,13,12,ID 04631182,TW0002301009,...,Electronics,M23D,電子零組件,Electron Component,M23D3,電源供應器,Power Supply Unit,,,


In [28]:
tse_data = data[['tseid', 'tejinm4_c']]
tse_data.columns = ['company_id', 'TSE']
tse_data

,company_id,TSE
None,,
0,000116,金融業
1,000930,金融業
2,000960,金融業
3,0009A0,金融業
4,1101,水泥工業
...,...,...
3147,9962,鋼鐵工業
3148,9965,航運業
3149,L2301,


In [49]:
tse_data.to_csv('data/TSE.csv', index=False)

In [30]:
tse = pd.merge(marketCap, tse_data, on='company_id', how="left").dropna()
tse

,company_id,date,acc_code,marketCap,TSE
0,1101,2008-03-01 00:00:00+00:00,MV,191344719.0,水泥工業
1,1101,2008-06-01 00:00:00+00:00,MV,133194117.0,水泥工業
2,1101,2008-09-01 00:00:00+00:00,MV,61399064.0,水泥工業
3,1101,2008-12-01 00:00:00+00:00,MV,88724116.0,水泥工業
4,1101,2009-03-01 00:00:00+00:00,MV,92016291.0,水泥工業
...,...,...,...,...,...
100623,9962,2022-03-01 00:00:00+00:00,MV,1863043.0,鋼鐵工業
100624,9962,2022-06-01 00:00:00+00:00,MV,1168349.0,鋼鐵工業
100625,9962,2022-09-01 00:00:00+00:00,MV,1326234.0,鋼鐵工業
100626,9965,2008-06-01 00:00:00+00:00,MV,799100.0,航運業


In [31]:
tse["date"] = pd.to_datetime(tse["date"]).dt.date


In [37]:
df_m = tse.pivot_table(index='date',columns='company_id')
df_m.columns = df_m.columns.get_level_values(1)

In [38]:
tseDict = tse_data.drop_duplicates(subset=['company_id'], keep='last')[['company_id', 'TSE']]
tseDict = tseDict.sort_values('company_id')
tseDict

,company_id,TSE
None,,
0,000116,金融業
1937,000121,金融業
1928,000572,金融業
1933,000582,金融業
1,000930,金融業
...,...,...
3145,9960,貿易百貨
3146,9961,貿易百貨
3147,9962,鋼鐵工業


In [39]:
df_weight = pd.DataFrame()
for ind in [i for i in tseDict.TSE.unique()]:
    lst = tseDict[tseDict['TSE']==ind]['company_id'].tolist()
    data = df_m[df_m.columns.intersection(lst)]
    data = data.div(data.sum(axis=1), axis=0)
    df_weight = pd.concat([df_weight, data], axis=1)

In [45]:
df_weight = df_weight.ffill().fillna(0)

In [46]:
df_weight.to_csv("output/rriTSEWeight.csv")

In [47]:
df_weight

company_id,2801,2809,2812,2816,2820,2823,2832,2833,2834,2836,...,911626,911868,912000,912398,9136,913889,9151,9157,916665,9188
date,,,,,,,,,,,,,,,,,,,,,
2008-03-01,0.032397,0.003183,0.006028,0.000886,0.003991,0.007631,0.002883,0.007884,0.013416,0.002645,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2008-06-01,0.033632,0.003230,0.005425,0.000813,0.003842,0.007940,0.002903,0.005961,0.014296,0.002306,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2008-09-01,0.037359,0.003920,0.005575,0.000777,0.003715,0.007897,0.002693,0.004926,0.014590,0.002260,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2008-12-01,0.043651,0.003337,0.005078,0.002489,0.004444,0.007688,0.002428,0.004827,0.014934,0.002004,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2009-03-01,0.041499,0.003445,0.004986,0.002367,0.007164,0.008431,0.002712,0.005415,0.015056,0.002300,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2009-06-01,0.036495,0.003113,0.004265,0.002918,0.006862,0.008262,0.002394,0.006964,0.012660,0.002164,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2009-09-01,0.033706,0.003118,0.004059,0.002374,0.005665,0.010967,0.003020,0.006140,0.011917,0.001954,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2009-12-01,0.033148,0.003378,0.004015,0.002395,0.005383,0.013060,0.002995,0.008638,0.011268,0.001923,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2010-03-01,0.034166,0.003611,0.004288,0.002255,0.006094,0.014055,0.003348,0.009428,0.012418,0.002210,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
